In [1]:
# 필요한 라이브러리 임포트
import cv2
import mediapipe as mp
import pyautogui


In [2]:
# MediaPipe 핸드 트래킹 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)
mp_draw = mp.solutions.drawing_utils


In [3]:
# 비디오 캡처 설정
cap = cv2.VideoCapture(0)

# 화면 크기 가져오기
screen_width, screen_height = pyautogui.size()


In [4]:
# 핸드 트래킹 루프 시작
while True:
    # 카메라에서 프레임 읽기
    success, img = cap.read()
    if not success:
        break

    # BGR 이미지를 RGB로 변환
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # 손 인식 처리
    result = hands.process(img_rgb)

    # 손 랜드마크 시각화 및 제어
    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_draw.draw_landmarks(img, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # 손가락 끝 점 좌표 가져오기 (여기서는 엄지 손가락)
            thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
            x = int(thumb_tip.x * screen_width)
            y = int(thumb_tip.y * screen_height)

            # 화면에 커서 위치 업데이트
            pyautogui.moveTo(x, y)

            # 특정 제스처로 클릭 (여기서는 엄지와 검지를 서로 접촉할 때 클릭)
            index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            if (abs(thumb_tip.x - index_tip.x) < 0.05) and (abs(thumb_tip.y - index_tip.y) < 0.05):
                pyautogui.click()

    # 결과 이미지 표시
    cv2.imshow("Hand Tracking", img)

    # 'q' 키를 눌러 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


In [5]:
# 리소스 해제
cap.release()
cv2.destroyAllWindows()
